# Interaction Model Examples

This notebook walks through the various interactions that can be initiated from the REPL towards a target using the Matter Interaction Model (IM) and Data Model (DM).

## Imports

Let's first begin by setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

In [1]:
from rich import print
from rich.pretty import pprint
from rich import inspect
import logging
import chip
import chip.logging
from chip import ChipDeviceCtrl
import chip.clusters as Clusters
from chip.ChipReplStartup import *
from chip.ChipStack import *
import subprocess, sys

──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

2022-01-18 14:07:57 johnsj-macbookpro1.roam.corp.google.com root[52410] CRITICAL Loading configuration from /tmp/repl-storage.json...


Loading CHIP DLL...


...

New FabricAdmin: FabricId: 1(1)


Fabric Admins have been loaded and are available at fabricAdmins

Creating default device controller on fabric 1...

Allocating new controller with FabricId: 1(1), NodeId: 1


Default CHIP Device Controller has been initialized to manage fabricAdmins[0], and is 
available as devCtrl

## Initialization

Next, let's initialize the REPL environment and the Matter Stack.

In [3]:
ReplInit()
chipStack = ChipStack()



──────────────────────────────────────── Matter REPL ────────────────────────────────────────

    
            Welcome to the Matter Python REPL!
    
            For help, please type matterhelp()
    
            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.
    
            

─────────────────────────────────────────────────────────────────────────────────────────────

## Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [6]:
subprocess.Popen(['pkill', '-f', 'chip-all-clusters-app'])
process = subprocess.Popen('./out/debug/chip-all-clusters-app', stdout=subprocess.DEVNULL)

### Commission Target

Commission the target with a NodeId of 1.

In [7]:
devCtrl = ChipDeviceCtrl.ChipDeviceController(controllerNodeId=1)
devCtrl.CommissionIP(b'127.0.0.1', 20202021, 1)

2022-01-02 21:25:51 johnsj-macbookpro1.roam.corp.google.com chip.CTL[93727] ERROR Unable to find country code, defaulting to WW


Established CASE with Device
Node address has been updated
Commissioning complete


True

## Invoke Commands

### Basic Command (Success Response)

Let's send a basic command to turn on/off the light on Endpoint 1.

In [8]:
await devCtrl.SendCommand(1, 1, Clusters.OnOff.Commands.On())

The receipt of a successful status response will result in the command just returning successfully. Otherwise, an exception will be thrown.

### Basic Command (Failure Response)

If we send the same command to an invalid endpoint, an exception is thrown:

In [9]:
await devCtrl.SendCommand(1, 100, Clusters.OnOff.Commands.On())

InteractionModelError: InteractionModelError: Failure (0x1)

### Basic Command (Data Response)

Here's an example of a command that sends back a data response, and how that is presented:

In [30]:
await devCtrl.SendCommand(1, 1, Clusters.TestCluster.Commands.TestListInt8UReverseRequest([1, 3, 5, 7]))

TestListInt8UReverseResponse(
│   arg1=[
│   │   7,
│   │   5,
│   │   3,
│   │   1
│   ]
)

### Read out the Fabric List to validate

In [5]:
await devCtrl.ReadAttribute(1, [(Clusters.OperationalCredentials)])

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.FabricsList'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04b\xeeV\x80\x01\xee\xab1\x0f|\x01\xc1\xaf5\x14\xf0\x18\x8f\xa2\xa3\xa8\x8b \x0c\x8c\xd5\xb0\xf6]BN\rS=\xa4U\xeb\xf5\x82\xbbdKU\xc8\xe9\x16\xeao}\xcc\xce<\xbf^\xd5\xa5\x06,L\x93\x1dX\xc5\xfc',
│   │   │   │   │   vendorId=26464,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=1,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.SupportedFabrics'>: 16,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.CommissionedFabrics'>: 1,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.TrustedRootCertificates'>: [
│   │   │   │   b'\x150\x01\x01\x00$\x02\x017\x03$\x14\x00$\x15\x01\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x14\x00$\x15\x01\x18$\x07\x01$\x08\x010\tA\x04b\xeeV\x80\x01\xee\xab1\x0f|\x01\xc1\xaf5\x14\xf0\x18\x8f\xa2\xa3\xa8\x8b \x0c\x8c\xd5\xb0\xf6]BN\rS=\xa4U\xeb\xf5\x82\xbbdKU\xc8\xe9\x16\xeao}\xcc\xce<\xbf^\xd5\xa5\x06,L\x93\x1dX\xc5\xfc7\n5\x01)\x01\x18$\x02`0\x04\x14q9\x96)~h\xbc\x04:\xc5\x12\xeb\xa8\x16y\xd4\xb2C\xba\xf50\x05\x14q9\x96)~h\xbc\x04:\xc5\x12\xeb\xa8\x16y\xd4\xb2C\xba\xf5\x180\x0b@\x13K\x909\xd6\xf2\xbb\x1c\\R/E\xe7\x00y\xbf^d8\x05\x89\x8c\xbeK\x14O\xac\x8c\xd44\x93\n\xb1\xe6k\xa7\x9c\xdc\xc5\xffR,\xcc\xb5\xc4\x84\x01\x92\x9d.\x9b\xda\x0f\x1d\xa2\xccEz-\x13\x05\x12\x1d\xcb\x18'
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.CurrentFabricIndex'>: 1,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.ClusterRevision'>: 1
│   │   }
│   }
}

In [27]:
await devCtrl.ReadAttribute(1, [(Clusters.Descriptor)], True)

{
│   0: {
│   │   <class 'chip.clusters.Objects.Descriptor'>: Descriptor(
│   │   │   deviceList=[
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   serverList=[
│   │   │   │   3,
│   │   │   │   4,
│   │   │   │   29,
│   │   │   │   30,
│   │   │   │   31,
│   │   │   │   40,
│   │   │   │   41,
│   │   │   │   42,
│   │   │   │   46,
│   │   │   │   48,
│   │   │   │   49,
│   │   │   │   50,
│   │   │   │   51,
│   │   │   │   52,
│   │   │   │   53,
│   │   │   │   54,
│   │   │   │   55,
│   │   │   │   60,
│   │   │   │   62,
│   │   │   │   63,
│   │   │   │   64,
│   │   │   │   65,
│   │   │   │   1029
│   │   │   ],
│   │   │   clientList=[],
│   │   │   partsList=[
│   │   │   │   1,
│   │   │   │   2
│   │   │   ],
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   )
│   },
│   1: {
│   │   <class 'chip.clusters.Objects.Descriptor'>: Descriptor(
│   │   │   deviceList=[
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   serverList=[
│   │   │   │   3,
│   │   │   │   4,
│   │   │   │   5,
│   │   │   │   6,
│   │   │   │   7,
│   │   │   │   8,
│   │   │   │   15,
│   │   │   │   29,
│   │   │   │   30,
│   │   │   │   37,
│   │   │   │   47,
│   │   │   │   57,
│   │   │   │   59,
│   │   │   │   64,
│   │   │   │   65,
│   │   │   │   69,
│   │   │   │   80,
│   │   │   │   257,
│   │   │   │   258,
│   │   │   │   259,
│   │   │   │   512,
│   │   │   │   513,
│   │   │   │   516,
│   │   │   │   768,
│   │   │   │   1024,
│   │   │   │   1026,
│   │   │   │   1027,
│   │   │   │   1028,
│   │   │   │   1029,
│   │   │   │   1030,
│   │   │   │   1280,
│   │   │   │   1283,
│   │   │   │   1284,
│   │   │   │   1285,
│   │   │   │   1286,
│   │   │   │   1287,
│   │   │   │   1288,
│   │   │   │   1289,
│   │   │   │   1290,
│   │   │   │   1291,
│   │   │   │   1292,
│   │   │   │   1293,
│   │   │   │   1294,
│   │   │   │   1295,
│   │   │   │   2820
│   │   │   ],
│   │   │   clientList=[],
│   │   │   partsList=[],
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   )
│   },
│   2: {
│   │   <class 'chip.clusters.Objects.Descriptor'>: Descriptor(
│   │   │   deviceList=[
│   │   │   │   DeviceType(
│   │   │   │   │   type=22,
│   │   │   │   │   revision=1
│   │   │   │   )
│   │   │   ],
│   │   │   serverList=[
│   │   │   │   4,
│   │   │   │   6,
│   │   │   │   29,
│   │   │   │   1030
│   │   │   ],
│   │   │   clientList=[],
│   │   │   partsList=[],
│   │   │   attributeList=None,
│   │   │   featureMap=None,
│   │   │   clusterRevision=1
│   │   )
│   }
}

Note that the `FabricsList` contains just a single item with a `fabriIndex` of 1, and a `fabricId` of 1.

### Create Controller on Fabric B

In [23]:
a = [1, 2, 3, 4]
a

[
│   1,
│   2,
│   3,
│   4
]

### Open Commissioning Window

The target right now doesn't accept commissioning requests. So let's go ahead and open the commissioning window to permit the second controller on Fabric B to commission the target. This request has to originate from the 1st controller.

In [7]:
await devCtrl.SendCommand(1, 0, Clusters.AdministratorCommissioning.Commands.OpenBasicCommissioningWindow(100))

### Commission Target on Fabric B

In [8]:
devCtrl2.CommissionIP(b'127.0.0.1', 20202021, 1)

2021-12-30 21:35:59 johnsj-macbookpro1.roam.corp.google.com chip.CTL[58883] ERROR Unable to find country code, defaulting to WW


Established CASE with Device


True

Node address has been updated
Commissioning complete


### Read out the Fabric List to validate

In [9]:
await devCtrl2.ReadAttribute(1, [(Clusters.OperationalCredentials)])

{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.FabricsList'>: [
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=1,
│   │   │   │   │   rootPublicKey=b'\x04b\xeeV\x80\x01\xee\xab1\x0f|\x01\xc1\xaf5\x14\xf0\x18\x8f\xa2\xa3\xa8\x8b \x0c\x8c\xd5\xb0\xf6]BN\rS=\xa4U\xeb\xf5\x82\xbbdKU\xc8\xe9\x16\xeao}\xcc\xce<\xbf^\xd5\xa5\x06,L\x93\x1dX\xc5\xfc',
│   │   │   │   │   vendorId=26464,
│   │   │   │   │   fabricId=1,
│   │   │   │   │   nodeId=1,
│   │   │   │   │   label=''
│   │   │   │   ),
│   │   │   │   FabricDescriptor(
│   │   │   │   │   fabricIndex=2,
│   │   │   │   │   rootPublicKey=b'\x04G+A\xc6\x15\xb0\xa6\x1e\xb2_J\x8e\xb1b\xc6\x16\xcd\xa8\xe63\x98\xb8\x7fr\x9a\xfe\x01#\xe0\x8fw4\xc1[\x17\xd1\x81.\xa0\r\x90\xef|\xf2\xe3\xe3\xf6\x82\x91@U\x90N\xbd\xdb\xb2 h\r\x10w\xf69\x93',
│   │   │   │   │   vendorId=26464,
│   │   │   │   │   fabricId=2,
│   │   │   │   │   nodeId=1,
│   │   │   │   │   label=''
│   │   │   │   )
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.SupportedFabrics'>: 16,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.CommissionedFabrics'>: 2,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.TrustedRootCertificates'>: [
│   │   │   │   b'\x150\x01\x01\x00$\x02\x017\x03$\x14\x00$\x15\x01\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x14\x00$\x15\x01\x18$\x07\x01$\x08\x010\tA\x04b\xeeV\x80\x01\xee\xab1\x0f|\x01\xc1\xaf5\x14\xf0\x18\x8f\xa2\xa3\xa8\x8b \x0c\x8c\xd5\xb0\xf6]BN\rS=\xa4U\xeb\xf5\x82\xbbdKU\xc8\xe9\x16\xeao}\xcc\xce<\xbf^\xd5\xa5\x06,L\x93\x1dX\xc5\xfc7\n5\x01)\x01\x18$\x02`0\x04\x14q9\x96)~h\xbc\x04:\xc5\x12\xeb\xa8\x16y\xd4\xb2C\xba\xf50\x05\x14q9\x96)~h\xbc\x04:\xc5\x12\xeb\xa8\x16y\xd4\xb2C\xba\xf5\x180\x0b@\x13K\x909\xd6\xf2\xbb\x1c\\R/E\xe7\x00y\xbf^d8\x05\x89\x8c\xbeK\x14O\xac\x8c\xd44\x93\n\xb1\xe6k\xa7\x9c\xdc\xc5\xffR,\xcc\xb5\xc4\x84\x01\x92\x9d.\x9b\xda\x0f\x1d\xa2\xccEz-\x13\x05\x12\x1d\xcb\x18',
│   │   │   │   b'\x150\x01\x01\x00$\x02\x017\x03$\x14\x00$\x15\x02\x18&\x04\x80"\x81\'&\x05\x80%M:7\x06$\x14\x00$\x15\x02\x18$\x07\x01$\x08\x010\tA\x04G+A\xc6\x15\xb0\xa6\x1e\xb2_J\x8e\xb1b\xc6\x16\xcd\xa8\xe63\x98\xb8\x7fr\x9a\xfe\x01#\xe0\x8fw4\xc1[\x17\xd1\x81.\xa0\r\x90\xef|\xf2\xe3\xe3\xf6\x82\x91@U\x90N\xbd\xdb\xb2 h\r\x10w\xf69\x937\n5\x01)\x01\x18$\x02`0\x04\x14\xc6\xb8;\xbf"u\xb8f"\x99\x10\xc5o\xdf\xc4(\x1b\xbd\xeeh0\x05\x14\xc6\xb8;\xbf"u\xb8f"\x99\x10\xc5o\xdf\xc4(\x1b\xbd\xeeh\x180\x0b@\tQ;Y\xb6\x82I[\x85k\xfdot\xb6\x98\x04\x1d\xf9SJ\xf74\xfa\xbc\xa1OWM\xef\xec\xcf|\xad[\x17\xd5/\xa9\xbb\xb4\xbc\xb1\xafa\x06\xdb\xcd]E\xdf\x84\xcf\xb6\x10\xd9\xc7\xf5l\xef\x96K6\xbd\xa5\x18'
│   │   │   ],
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.CurrentFabricIndex'>: 2,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.ClusterRevision'>: 1
│   │   }
│   }
}

Note that the FabricsList contains two items now!